# InceptionV3 Mode

In [42]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import InceptionV3
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np

In [43]:
# Define paths to your training, validation, and test sets
train_dir = 'C:/Users/user/Downloads/archive (3)/train_set'
validation_dir = 'C:/Users/user/Downloads/archive (3)/validation_set'
test_dir = 'C:/Users/user/Downloads/archive (3)/test_set'

In [44]:
# Data preprocessing and augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(299, 299),
    batch_size=32,
    class_mode='binary'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(299, 299),
    batch_size=32,
    class_mode='binary'
)

Found 452 images belonging to 2 classes.
Found 94 images belonging to 2 classes.


In [45]:
# Load the pre-trained InceptionV3 model
base_model = InceptionV3(weights='imagenet', include_top=False)

In [46]:
# Add custom classification layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

In [47]:
# Compile the model
model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer=Adam(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

In [48]:
# Train the model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)

Epoch 1/10
14/14 [==============================] - 200s 11s/step - loss: 0.7092 - accuracy: 0.5571 - val_loss: 1.0002 - val_accuracy: 0.4062
Epoch 2/10
14/14 [==============================] - 134s 9s/step - loss: 0.5338 - accuracy: 0.7214 - val_loss: 0.7123 - val_accuracy: 0.6250
Epoch 3/10
14/14 [==============================] - 134s 9s/step - loss: 0.3934 - accuracy: 0.8452 - val_loss: 0.5632 - val_accuracy: 0.7031
Epoch 4/10
14/14 [==============================] - 129s 9s/step - loss: 0.3001 - accuracy: 0.8738 - val_loss: 0.5904 - val_accuracy: 0.6875
Epoch 5/10
14/14 [==============================] - 129s 9s/step - loss: 0.3093 - accuracy: 0.8548 - val_loss: 0.7318 - val_accuracy: 0.6406
Epoch 6/10
14/14 [==============================] - 130s 9s/step - loss: 0.3657 - accuracy: 0.8357 - val_loss: 0.8000 - val_accuracy: 0.6875
Epoch 7/10
14/14 [==============================] - 129s 9s/step - loss: 0.2385 - accuracy: 0.9000 - val_loss: 0.9294 - val_accuracy: 0.6094
Epoch 8/10
1

In [49]:
# Evaluate the model on the test set
test_generator = validation_datagen.flow_from_directory(
    test_dir,
    target_size=(299, 299),
    batch_size=32,
    class_mode='binary'
)

Found 94 images belonging to 2 classes.


In [50]:
test_loss, test_accuracy = model.evaluate(test_generator)
print("Test Accuracy:", test_accuracy)

3/3 [==============================] - 8s 3s/step - loss: 0.6479 - accuracy: 0.6915
Test Accuracy: 0.6914893388748169


## Below is fine tuning of the model

In [51]:
# Unfreeze some top layers for fine-tuning
for layer in model.layers[:249]:
    layer.trainable = False
for layer in model.layers[249:]:
    layer.trainable = True

In [52]:
# Compile the model again for fine-tuning
model.compile(optimizer=Adam(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

In [53]:
# Continue training the entire model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10,  # You can adjust the number of epochs as needed
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)

Epoch 1/10
14/14 [==============================] - 64s 4s/step - loss: 0.2715 - accuracy: 0.8795 - val_loss: 0.7938 - val_accuracy: 0.6562
Epoch 2/10
14/14 [==============================] - 56s 4s/step - loss: 0.2127 - accuracy: 0.8929 - val_loss: 1.2775 - val_accuracy: 0.7031
Epoch 3/10
14/14 [==============================] - 55s 4s/step - loss: 0.2342 - accuracy: 0.9119 - val_loss: 1.3075 - val_accuracy: 0.6562
Epoch 4/10
14/14 [==============================] - 50s 4s/step - loss: 0.1881 - accuracy: 0.9190 - val_loss: 1.3156 - val_accuracy: 0.6719
Epoch 5/10
14/14 [==============================] - 50s 4s/step - loss: 0.1521 - accuracy: 0.9357 - val_loss: 1.2882 - val_accuracy: 0.6875
Epoch 6/10
14/14 [==============================] - 50s 3s/step - loss: 0.1743 - accuracy: 0.9381 - val_loss: 1.6144 - val_accuracy: 0.6562
Epoch 7/10
14/14 [==============================] - 49s 3s/step - loss: 0.1380 - accuracy: 0.9500 - val_loss: 1.4513 - val_accuracy: 0.6406
Epoch 8/10
14/14 [==

In [54]:
# Evaluate the model on the validation set
validation_loss, validation_accuracy = model.evaluate(validation_generator)
print("Validation Accuracy:", validation_accuracy)

3/3 [==============================] - 7s 2s/step - loss: 1.1670 - accuracy: 0.7340
Validation Accuracy: 0.7340425252914429


In [55]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_generator)
print("Test Accuracy:", test_accuracy)

3/3 [==============================] - 8s 2s/step - loss: 1.1590 - accuracy: 0.6809
Test Accuracy: 0.6808510422706604


### Testing with images

In [56]:
# Classify a single image
def classify_image(img_path):
    img = load_img(img_path, target_size=(299, 299))
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.  # Rescale pixel values to [0, 1]
    
    prediction = model.predict(img_array)
    
    if prediction[0] >= 0.5:
        return "Pneumonia"
    else:
        return "Non-Pneumonia"

In [63]:
# Example usage
image_path_to_classify = r'C:\Users\user\Downloads\archive (3)\Pneumonia photos\00030621_002.png'
classification_result = classify_image(image_path_to_classify)
print("Classification Result:", classification_result)

1/1 [==============================] - 0s 77ms/step
Classification Result: Pneumonia


In [64]:
# Example usage
image_path_to_classify = r'C:\Users\user\Downloads\archive (3)\Non-pneumonia photos\00000078_000.png'
classification_result = classify_image(image_path_to_classify)
print("Classification Result:", classification_result)

1/1 [==============================] - 0s 78ms/step
Classification Result: Non-Pneumonia


# Tensorflow

In [65]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image

In [66]:
# Data preprocessing and augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [67]:
validation_datagen = ImageDataGenerator(rescale=1./255)

In [68]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),  # Adjust target size as needed
    batch_size=32,
    class_mode='binary'
)

Found 452 images belonging to 2 classes.


In [69]:
validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),  # Adjust target size as needed
    batch_size=32,
    class_mode='binary'
)

Found 94 images belonging to 2 classes.


In [70]:
# Define the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(1, activation='sigmoid')
])

In [71]:
# Compile the model
model.compile(optimizer=Adam(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

In [72]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)

Epoch 1/10
14/14 [==============================] - 12s 844ms/step - loss: 0.6977 - accuracy: 0.5190 - val_loss: 0.6951 - val_accuracy: 0.4844
Epoch 2/10
14/14 [==============================] - 12s 841ms/step - loss: 0.6872 - accuracy: 0.5452 - val_loss: 0.6970 - val_accuracy: 0.5156
Epoch 3/10
14/14 [==============================] - 14s 993ms/step - loss: 0.6981 - accuracy: 0.5143 - val_loss: 0.6908 - val_accuracy: 0.5625
Epoch 4/10
14/14 [==============================] - 13s 960ms/step - loss: 0.6934 - accuracy: 0.5119 - val_loss: 0.6879 - val_accuracy: 0.5938
Epoch 5/10
14/14 [==============================] - 13s 953ms/step - loss: 0.6892 - accuracy: 0.5310 - val_loss: 0.6896 - val_accuracy: 0.5781
Epoch 6/10
14/14 [==============================] - 13s 968ms/step - loss: 0.6851 - accuracy: 0.5476 - val_loss: 0.6851 - val_accuracy: 0.5312
Epoch 7/10
14/14 [==============================] - 14s 992ms/step - loss: 0.6930 - accuracy: 0.5381 - val_loss: 0.6927 - val_accuracy: 0.5000

In [73]:
# Evaluate the model on the test set
test_generator = validation_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),  # Adjust target size as needed
    batch_size=32,
    class_mode='binary'
)

Found 94 images belonging to 2 classes.


In [74]:
test_loss, test_accuracy = model.evaluate(test_generator)
print("Test Accuracy:", test_accuracy)

3/3 [==============================] - 1s 477ms/step - loss: 0.6910 - accuracy: 0.5426
Test Accuracy: 0.542553186416626


## Below is fine tuning of the model

In [75]:
# Data preprocessing and augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [76]:
validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(299, 299),
    batch_size=32,
    class_mode='binary'
)


Found 452 images belonging to 2 classes.


In [77]:
validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(299, 299),
    batch_size=32,
    class_mode='binary'
)

# Define your custom model architecture
base_model = InceptionV3(weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Freeze all layers except the last few
for layer in base_model.layers[:-10]:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

Found 94 images belonging to 2 classes.


In [78]:
# Train the model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)

Epoch 1/10
14/14 [==============================] - 40s 3s/step - loss: 0.6511 - accuracy: 0.6405 - val_loss: 0.7559 - val_accuracy: 0.5000
Epoch 2/10
14/14 [==============================] - 42s 3s/step - loss: 0.5737 - accuracy: 0.7024 - val_loss: 0.6468 - val_accuracy: 0.5938
Epoch 3/10
14/14 [==============================] - 44s 3s/step - loss: 0.5479 - accuracy: 0.7310 - val_loss: 0.6351 - val_accuracy: 0.6250
Epoch 4/10
14/14 [==============================] - 45s 3s/step - loss: 0.5143 - accuracy: 0.7786 - val_loss: 0.7033 - val_accuracy: 0.6406
Epoch 5/10
14/14 [==============================] - 45s 3s/step - loss: 0.5117 - accuracy: 0.7286 - val_loss: 0.6377 - val_accuracy: 0.6875
Epoch 6/10
14/14 [==============================] - 44s 3s/step - loss: 0.5223 - accuracy: 0.7286 - val_loss: 0.6380 - val_accuracy: 0.6406
Epoch 7/10
14/14 [==============================] - 45s 3s/step - loss: 0.4912 - accuracy: 0.7762 - val_loss: 0.5646 - val_accuracy: 0.7344
Epoch 8/10
14/14 [==

In [79]:
# Evaluate the model on the test set
test_generator = validation_datagen.flow_from_directory(
    test_dir,
    target_size=(299, 299),
    batch_size=32,
    class_mode='binary'
)

Found 94 images belonging to 2 classes.


In [80]:
test_loss, test_accuracy = model.evaluate(test_generator)
print("Test Accuracy:", test_accuracy)

3/3 [==============================] - 5s 2s/step - loss: 0.6630 - accuracy: 0.6489
Test Accuracy: 0.6489361524581909


### Testing with images

In [81]:
# Function to classify a single image
def classify_single_image(img_path):
    # Load the image
    img = image.load_img(img_path, target_size=(150, 150))
    # Convert image to array
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    # Normalize pixel values
    img_array /= 255.

    # Predict class
    prediction = model.predict(img_array)
    if prediction[0] >= 0.5:
        return "Pneumonia"
    else:
        return "Non-Pneumonia"

In [82]:
# Example usage
image_path_to_classify = r'C:\Users\user\Downloads\archive (3)\Pneumonia photos\00030621_002.png'
classification_result = classify_single_image(image_path_to_classify)
print("Classification Result:", classification_result)

1/1 [==============================] - 1s 1s/step
Classification Result: Pneumonia


In [84]:
# Example usage
image_path_to_classify = r'C:\Users\user\Downloads\archive (3)\Non-pneumonia photos\00000013_010.png'
classification_result = classify_image(image_path_to_classify)
print("Classification Result:", classification_result)

1/1 [==============================] - 0s 102ms/step
Classification Result: Non-Pneumonia
